In [9]:
from model_classes import *
import torch
import torch.nn as nn
from transformers import PretrainedConfig

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Wszystkie informacje o modelu zawarte są w obiekcie config z hugging face'owej klasy Pretrained Config. Kiedy twożymy nowy transformer, podajemy mu jako argument tylko config. Musi on mieć parametr 'forward_layer_class' i to ma być klasa, którą chcemy mieć w transformerze (tzn lori albo moe itd). Ta klasa sama też ma dostawać tylko parametr config, więc ten config musi mieć wszystkie parametry potrzebne do funkcjonowania tej klasy. W kodzie klas używałem takich nazw parametrów jak w configu poniżej więc proszę ich nie zmieniać.

In [11]:
config = PretrainedConfig(
    num_experts_per_token=2,
    hidden_size=512,
    num_attention_heads = 16,
    num_experts=32,
    batch_size = 16,
    seq_len = 32,
    capacity_factor = 8,
    device = device,
    intermediate_size = 1024,
    forward_layer_class = VectorizedMoE,
    vocab_size = 1000,
    n_layers = 8
)

In [12]:
example_input = torch.rand((config.batch_size, config.seq_len, config.hidden_size))

TransformerBlock_test = TransformerBlock(config)
example_output = TransformerBlock_test(example_input)

print(example_output.shape)

torch.Size([16, 32, 512])


In [13]:
example_input_tokens = torch.randint(low = 0, high = config.vocab_size, size = (config.batch_size, config.seq_len))

Transformer_test = Transformer(config)
example_output_tokens = Transformer_test(example_input_tokens)

print(example_output_tokens.shape)

torch.Size([16, 32, 1000])


Tutaj sprawdzam czy model na pewno się dobrze zainicjował i czy różne warstwy mają różne parametry i wychodzi że tak

In [14]:
l1 = Transformer_test.layers[0].forward_layer.first_linear
l2 = Transformer_test.layers[4].forward_layer.first_linear

a1 = Transformer_test.layers[0].attention.projector.query_transform.state_dict()['weight']
a2 = Transformer_test.layers[4].attention.projector.query_transform.state_dict()['weight']

print(torch.equal(l1, l2))
print(torch.equal(a1, a2))

False
False
